In [3]:
import numpy as np
from PIL import Image
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.image import ssim_multiscale
from tensorflow.nn import relu,depth_to_space
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, Input, Add, Lambda,UpSampling2D, Dense, GlobalAveragePooling2D

Loading data

In [2]:
lr_dir = "/kaggle/input/img-superres/AnalyticsArena_DataSet/lowres"
hr_dir = "/kaggle/input/img-superres/AnalyticsArena_DataSet/highres"

lr_datagen = tf.keras.preprocessing.image.ImageDataGenerator()

hr_datagen = tf.keras.preprocessing.image.ImageDataGenerator()

batch_size = 32
target_size = (510, 510)

lr_iterator = lr_datagen.flow_from_directory(
    directory=lr_dir,
    target_size=(170,170),
    batch_size=1,
    class_mode=None,
    shuffle=False
)

hr_iterator = hr_datagen.flow_from_directory(
    directory=hr_dir,
    target_size=(510,510),
    batch_size=1,
    class_mode=None,
    shuffle=False
)
iterator = zip(lr_iterator, hr_iterator)

Found 3500 images belonging to 1 classes.
Found 3500 images belonging to 1 classes.


In [3]:
lrv_dir = "/kaggle/input/img-superres/AnalyticsArena_DataSet/lowresvalid"
hrv_dir = "/kaggle/input/img-superres/AnalyticsArena_DataSet/highresvalid"

lrv_datagen = tf.keras.preprocessing.image.ImageDataGenerator()

hrv_datagen = tf.keras.preprocessing.image.ImageDataGenerator()

batch_size = 32
target_size = (510, 510)

lrv_iterator = lrv_datagen.flow_from_directory(
    directory=lrv_dir,
    target_size=(170,170),
    batch_size=1,
    class_mode=None,
    shuffle=False
)

hrv_iterator = hrv_datagen.flow_from_directory(
    directory=hrv_dir,
    target_size=(510,510),
    batch_size=1,
    class_mode=None,
    shuffle=False
)
iteratorv = zip(lrv_iterator, hrv_iterator)

Found 1000 images belonging to 1 classes.
Found 1000 images belonging to 1 classes.


In [4]:
def ms_ssim(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32) / 255.0
    y_pred = tf.cast(y_pred, tf.float32) / 255.0
    ms_ssim = tf.reduce_mean(ssim_multiscale(y_true, y_pred, max_val=1.0))
    loss = 1 - ms_ssim
    return loss

In [8]:
def ResBlock(input_layer, filters):
    x = Conv2D(filters=filters, kernel_size=3, padding='same', activation='relu')(input_layer)
    x = Conv2D(filters=filters, kernel_size=3, padding='same')(x)
    x = Add()([input_layer, x])
    return x
def InterConnected(input_shape):
    input_tensor = Input(shape=input_shape)
    x = Conv2D(filters=32, kernel_size=3, padding='same', activation='relu')(input_tensor)
    for i in range(2):
        x = ResBlock(x, 32)
    x = Conv2D(filters=32*9, kernel_size=3, padding='same')(x)
    x = Lambda(lambda x: tf.nn.depth_to_space(x, block_size=3))(x)
    output_tensor = Conv2D(filters=3, kernel_size=1)(x)
    model = Model(inputs=input_tensor, outputs=output_tensor)
    print(model.summary())
    return model
model=InterConnected((170,170,3))

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 170, 170, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_21 (Conv2D)             (None, 170, 170, 32  896         ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_22 (Conv2D)             (None, 170, 170, 32  9248        ['conv2d_21[0][0]']              
                                )                                                           

In [6]:
model.compile(optimizer=Adam(lr=1e-5), loss='mse', metrics=[ms_ssim,'mse'])

In [7]:
model.fit(iterator,validation_data=iteratorv,epochs=20,steps_per_epoch=3500,verbose=1,validation_steps=750)

Epoch 1/20
3500/3500 [==============================] - 159s 40ms/step - loss: 0.0448 - ms_ssim: 0.0448 - mse: 131.3450 - val_loss: 0.0409 - val_ms_ssim: 0.0409 - val_mse: 87.2843
Epoch 2/20
3500/3500 [==============================] - 143s 41ms/step - loss: 0.0402 - ms_ssim: 0.0402 - mse: 88.8479 - val_loss: 0.0389 - val_ms_ssim: 0.0389 - val_mse: 74.7232
Epoch 3/20
3500/3500 [==============================] - 137s 39ms/step - loss: 0.0392 - ms_ssim: 0.0392 - mse: 85.7091 - val_loss: 0.0380 - val_ms_ssim: 0.0380 - val_mse: 72.1360
Epoch 4/20
3500/3500 [==============================] - 143s 41ms/step - loss: 0.0385 - ms_ssim: 0.0385 - mse: 80.9103 - val_loss: 0.0371 - val_ms_ssim: 0.0371 - val_mse: 70.2072
Epoch 5/20
3500/3500 [==============================] - 142s 41ms/step - loss: 0.0381 - ms_ssim: 0.0381 - mse: 78.6119 - val_loss: 0.0377 - val_ms_ssim: 0.0377 - val_mse: 75.2312
Epoch 6/20
3500/3500 [==============================] - 137s 39ms/step - loss: 0.0378 - ms_ssim: 0.0378 

KeyboardInterrupt: 

In [ ]:
model.save_weights("resblock1_2_32_20.h5")

In [ ]:
img=Image.open("/kaggle/input/img-superres/AnalyticsArena_DataSet/lowresvalid/LowResolution_3x_Valid/Img_down3503.jpg")
arr=np.array(img)
pred=model.predict(arr.reshape(1,170,170,3))
pred=pred.astype(np.uint8)
plt.imshow(pred[0])